In [1]:
import torch
import random
from seeker import White_seeker, Random_select_seeker, Random_gen_seeker, Black_seeker
from utils import Unfair_metric, load_model, get_L_matrix
from data import adult
from train_dnn import get_data
from dnn_models.model import MLP
from distances.normalized_mahalanobis_distances import SquaredEuclideanDistance, ProtectedSEDistances
from distances.sensitive_subspace_distances import LogisticRegSensitiveSubspace
from distances.binary_distances import BinaryDistance
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from IPython.display import display

%load_ext autoreload
%autoreload 2

In [2]:
rand_seed = 0
use_protected_attr = True
protected_vars = ['sex_Male']
# protected_vars = ['sex_Male', 'race_White']

dataset, train_dl, test_dl = get_data(adult, rand_seed, protected_vars=protected_vars)
dataset.use_protected_attr = use_protected_attr
in_dim = dataset.dim_feature()
out_dim = 2

model = MLP(in_dim, out_dim)
load_model(model, 'MLP', 'adult', 'STDTrainer', use_protected_attr=use_protected_attr, \
           protected_vars=protected_vars, id=rand_seed)

In [3]:
# prepare data
all_X, all_y = dataset.get_all_data(), dataset.labels
all_pred = model.get_prediction(all_X)

In [4]:
adult_gen = adult.Adult_gen(sensitive_columns=dataset.protected_idxs, include_protected_feature=use_protected_attr)

In [5]:
# prepare distances
distance_x_NSE = SquaredEuclideanDistance()
distance_x_Causal = ProtectedSEDistances()
distance_x_LR = LogisticRegSensitiveSubspace()
distance_y = BinaryDistance()

distance_x_NSE.fit(num_dims=dataset.dim_feature(), data_gen=adult_gen)
distance_x_Causal.fit(num_dims=dataset.dim_feature(), data_gen=adult_gen, protected_idx=dataset.protected_idxs)
distance_x_LR.fit(all_X, adult_gen, protected_idxs=dataset.protected_idxs)

In [6]:
def rand_gen():
    return {
        'age': random.randint(15, 60),
        'capital_gain': 0,
        'capital_loss': 0,
        'education_num': random.randint(1, 15),
        'hours_per_week': random.randint(10, 50),
        'race_white': random.choice([0, 1]),
        'sex_male': random.choice([0, 1]),
        'marital_status': random.choice(list(range(7))),
        'occupation': random.choice(list(range(14))),
        'relationship': random.choice(list(range(6))),
        'workclass': random.choice(list(range(7)))
    }

def perturb_pair(x, pert_features, pert_func):
    pair = dict()
    for k, v in x.items():
        pair[k] = torch.tensor([v, v])
        if k in pert_features:
            pair[k][1] = pert_func(pair[k][0])
    return pair

x = rand_gen()
# pair = adult.generate_from_origin(**perturb_pair(x, ['sex_male', 'race_white'], lambda x:1-x))
# adult.get_original_feature(pair)

In [7]:
pair = adult.generate_from_origin(**perturb_pair(x, ['capital_loss'], lambda x:1+x))
adult.get_original_feature(pair)

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,38.0,0.0,0.0,12.0,27.0,0.0,1.0,4.0,11.0,0.0,1.0
1,38.0,0.0,1.0,12.0,27.0,0.0,1.0,4.0,11.0,0.0,1.0


In [8]:
# dist = distance_x_LR(all_X[0], all_X[0] + torch.eye(all_X.shape[1]), itemwise_dist=False).squeeze()
# print(dist)
# print(dist[dataset.protected_idxs])

In [9]:
# 1/1e-10

In [10]:
dist = distance_x_Causal(all_X[0], all_X[0] + torch.eye(all_X.shape[1]), itemwise_dist=False).squeeze()
print(dist)
print(dist[dataset.protected_idxs])

tensor([1.8765e-04, 1.0001e-10, 5.2702e-08, 4.4444e-03, 1.0412e-04, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00])
tensor([0.])


In [11]:
# x = rand_gen()
# for key in x:
#     pair = adult.generate_from_origin(**perturb_pair(x, key, x[key]+1))
#     print(distance_x_NSE(pair[0], pair[1]), 1 / distance_x_NSE(pair[0], pair[1]))

In [12]:
# epsilon越大，要求不公平样本对dx越小，越严格
# epsilon = 1e10
epsilon = 9e9
unfair_metric = Unfair_metric(dx=distance_x_Causal, dy=distance_y, epsilon=epsilon)

In [13]:
L = get_L_matrix(all_X, all_pred, distance_x_Causal, distance_y)
n_unfair = torch.sum(L>epsilon).item()

total_pairs = all_X.shape[0]**2

unfair_ratio = n_unfair/total_pairs
unfair_ratio

  0%|          | 0/16 [00:00<?, ?it/s]

0.0

In [14]:
# 1/unfair_ratio

In [15]:
# p_success = lambda n: 1 - (1 - unfair_ratio)**n
# for i in range(10):
#     if p_success(10**i) > 1/100:
#         print(10**i, p_success(10**i))

In [16]:
def show_result(result):
    pair, n_query = result
    if pair != None:
        display(adult.get_original_feature(pair), n_query)
    else:
        print('not found')

In [17]:
# random.seed(422)
# torch.manual_seed(422)

# select_seeker = Random_select_seeker(model=model, unfair_metric=unfair_metric, data=all_X)
# for _ in range(5):
#     show_result(select_seeker.seek(dx_constraint=True, max_query=1e6))

In [18]:
# random.seed(422)
# torch.manual_seed(422)

# gen_seeker = Random_gen_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
# for _ in range(3):
#     show_result(gen_seeker.seek(by_range=True, max_query=1e6))

In [19]:
# random.seed(422)
# torch.manual_seed(422)

# for _ in range(3):
#     show_result(gen_seeker.seek(by_range=False, max_query=1e6))

In [20]:
adult_gen.get_range('data')

tensor([[1.7000e+01, 0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [9.0000e+01, 9.9999e+04, 4.3560e+03, 1.6000e+01, 9.9000e+01, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000

In [21]:
random.seed(422)
torch.manual_seed(422)

white_seeker = White_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
# show_result(white_seeker.seek())
for _ in range(5):
    show_result(white_seeker.seek(max_query=1e6))

1000000000000000.0
calculate gradient
loss tensor(19.9574, grad_fn=<SubBackward0>)
20480000.0
20480000.0
calculate gradient
loss tensor(8.7825, grad_fn=<SubBackward0>)
20480000.0
calculate gradient
loss tensor(3.9057, grad_fn=<SubBackward0>)
4096000.0
4096000.0
calculate gradient
loss tensor(2.8857, grad_fn=<SubBackward0>)
4096000.0
calculate gradient
loss tensor(1.8657, grad_fn=<SubBackward0>)
4096000.0
calculate gradient
loss tensor(0.8457, grad_fn=<SubBackward0>)
819200.0
819200.0
calculate gradient
loss tensor(0.6416, grad_fn=<SubBackward0>)
819200.0
calculate gradient
loss tensor(0.4375, grad_fn=<SubBackward0>)
819200.0
calculate gradient
loss tensor(0.2333, grad_fn=<SubBackward0>)
819200.0
calculate gradient
loss tensor(0.0292, grad_fn=<SubBackward0>)
163840.0
32768.0
32768.0
calculate gradient
loss tensor(0.0213, grad_fn=<SubBackward0>)
32768.0
calculate gradient
loss tensor(0.0133, grad_fn=<SubBackward0>)
32768.0
calculate gradient
loss tensor(0.0053, grad_fn=<SubBackward0>)
65

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,90.0,9448.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0
1,90.0,9447.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0


41

1000000000000000.0
calculate gradient
loss tensor(34.8165, grad_fn=<SubBackward0>)
102400000.0
102400000.0
calculate gradient
loss tensor(2.6037, grad_fn=<SubBackward0>)
20480000.0
4096000.0
4096000.0
calculate gradient
loss tensor(1.5837, grad_fn=<SubBackward0>)
4096000.0
calculate gradient
loss tensor(0.5637, grad_fn=<SubBackward0>)
819200.0
819200.0
calculate gradient
loss tensor(0.3596, grad_fn=<SubBackward0>)
819200.0
calculate gradient
loss tensor(0.1555, grad_fn=<SubBackward0>)
163840.0
163840.0
calculate gradient
loss tensor(0.1146, grad_fn=<SubBackward0>)
163840.0
calculate gradient
loss tensor(0.0737, grad_fn=<SubBackward0>)
163840.0
calculate gradient
loss tensor(0.0327, grad_fn=<SubBackward0>)
32768.0
32768.0
calculate gradient
loss tensor(0.0248, grad_fn=<SubBackward0>)
32768.0
calculate gradient
loss tensor(0.0168, grad_fn=<SubBackward0>)
32768.0
calculate gradient
loss tensor(0.0088, grad_fn=<SubBackward0>)
32768.0
calculate gradient
loss tensor(0.0008, grad_fn=<SubBackw

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,90.0,9448.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0
1,90.0,9447.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0


...

1000000000000000.0
calculate gradient
loss tensor(44.1338, grad_fn=<SubBackward0>)
102400000.0
102400000.0
calculate gradient
loss tensor(17.9934, grad_fn=<SubBackward0>)
20480000.0
20480000.0
calculate gradient
loss tensor(13.2992, grad_fn=<SubBackward0>)
20480000.0
calculate gradient
loss tensor(8.6049, grad_fn=<SubBackward0>)
20480000.0
calculate gradient
loss tensor(3.7206, grad_fn=<SubBackward0>)
4096000.0
4096000.0
calculate gradient
loss tensor(2.7005, grad_fn=<SubBackward0>)
4096000.0
calculate gradient
loss tensor(1.6805, grad_fn=<SubBackward0>)
4096000.0
calculate gradient
loss tensor(0.6605, grad_fn=<SubBackward0>)
819200.0
819200.0
calculate gradient
loss tensor(0.4564, grad_fn=<SubBackward0>)
819200.0
calculate gradient
loss tensor(0.2523, grad_fn=<SubBackward0>)
819200.0
calculate gradient
loss tensor(0.0482, grad_fn=<SubBackward0>)
163840.0
163840.0
calculate gradient
loss tensor(0.0073, grad_fn=<SubBackward0>)
32768.0
6553.6
6553.6
calculate gradient
loss tensor(0.0058,

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,90.0,9448.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0
1,90.0,9447.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0


44

1000000000000000.0
calculate gradient
loss tensor(4.8142, grad_fn=<SubBackward0>)
4096000.0
819200.0
819200.0
calculate gradient
loss tensor(3.1643, grad_fn=<SubBackward0>)
819200.0
calculate gradient
loss tensor(2.1659, grad_fn=<SubBackward0>)
163840.0
163840.0
calculate gradient
loss tensor(1.8559, grad_fn=<SubBackward0>)
163840.0
calculate gradient
loss tensor(1.4288, grad_fn=<SubBackward0>)
163840.0
calculate gradient
loss tensor(0.2893, grad_fn=<SubBackward0>)
32768.0
32768.0
calculate gradient
loss tensor(0.0516, grad_fn=<SubBackward0>)
6553.6
6553.6
calculate gradient
loss tensor(0.0017, grad_fn=<SubBackward0>)
1310.72
262.144
52.4288
tensor([[-0.0056,  0.0002,  0.0028, -0.0028, -0.0066, -0.0310,  0.0055,  0.0149,
         -0.0090, -0.0430, -0.0191, -0.0170, -0.0239,  0.0012, -0.0117,  0.0145,
         -0.0171, -0.0226, -0.0144, -0.0307, -0.0118,  0.0101,  0.0013, -0.0036,
          0.0033, -0.0122, -0.0097,  0.0163, -0.0392, -0.0183, -0.0325, -0.0318,
          0.0033, -0.0035,

Exception: lalala

In [ ]:
random.seed(422)
torch.manual_seed(422)

black_seeker = Black_seeker(model=model, unfair_metric=unfair_metric, data_gen=adult_gen)
for _ in range(5):
    show_result(black_seeker.seek(max_query=1e6))

calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
out1: directly find a unfair pair when processing gradiant descent


,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,90.0,9490.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0
1,90.0,9408.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0


834

calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
out1: directly find a unfair pair when processing gradiant descent


,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,90.0,9472.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0
1,90.0,9390.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0


...

,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,90.0,9506.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0
1,90.0,9424.0,0.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0


918

calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
calculate gradient
out1: directly find a unfair pair when processing gradiant descent


,age,capital-gain,capital-loss,education-num,hours-per-week,race_White,sex_Male,marital-status,occupation,relationship,workclass
0,90.0,4622.0,591.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0
1,90.0,4622.0,587.0,16.0,99.0,1.0,1.0,4.0,7.0,1.0,2.0


919